In [1]:
# valuation_date時におけるcodeで指定した銘柄の個別銘柄分析
# SBI証券のモバイルの銘柄分析を参考に作成

In [2]:
### パラメータ設定セル
from datetime import date
from dateutil.relativedelta import relativedelta

# パラメータをセット
# code: 証券コード
# valuation_date: 過去にさかのぼって評価日をdatetime.dateでセット可能。デフォルト値は今日(date.today())
code = 3930
valuation_date = date.today()
valuation_date = date(2024, 12, 13)


In [3]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, FinancequotePl, KessanPl, KessanFig, PricelistFig, read_data, get_df_latest_yearly_performance, get_latest_stockprice, print_finance_quote
from lib_dataprocess import get_fig_expected_performance_progress_rate_pycharts, MeigaralistPl, get_fig_actual_performance_progress_rate_pycharts, ShikihoPl
import polars as pl

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)
FPL.with_columns_market_cap()

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [4]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

はてな(3930)の銘柄情報

終値: 1000.0円(2024年12月13日)
予想配当利回り: None%(2024年12月13日)
予想PER: 21.11倍(2024年12月13日)
実績PBR: 1.15倍(2024年12月13日)
自己資本比率: 88.0%(2024年12月13日)
予想ROE: 5.47%(2024年12月13日)
予想ROA: 4.81%(2024年12月13日)
時価総額: 30.7億円(2024年12月13日)

3930(はてな)の四季報データ履歴

発行日: 2024年09月13日
【好転】
  25年7月期は受託サービスでビューワの『少年ジャンプ+』搭載が通期貢献。『はてなブログ』の運用型広告は苦戦続くが、法人向け案件増。開発人件費やデータセンター利用料増こなし、営業益好転。
【集英社】
  漫画アプリビューワ提供に加え広告運用やコンテンツ販売支援、専用アプリでの次世代ヒット作創出で協業。生成AI活用の企業向け発話分析サービス開始。

発行日: 2024年06月14日
【費用増】
  受託サービスは下期から集英社が『少年ジャンプ+』でアプリビューワ採用し貢献大。はてな関連は企業向けオウンドメディア支援下げ止まる。ただ人件費やデータセンター利用料の上昇重い。連続減益。25年7月期は受託サービスの収益軌道化。
【漫画アプリ】
  ブラウザーとの高い連動性をアピールし、集英社に続く大型案件獲得狙う。任天堂のスマホサービス開発に協力。

発行日: 2024年03月16日
【増収減益】
  受託サービスはWeb・アプリともに導入進展、前期ずれ込んだ大型漫画アプリ開発案件も寄与。『はてな』関連は個人向けの課金伸び悩む一方、広告単価復調。ただ受託中心に人員体制の拡充進めるほか、クラウド利用料等膨らみ営業益続落。
【人員体制】
  社員採用に加え、外注・業務委託活用。フルリモート勤務容認。生成AI使ったタイトル提案機能は法人にも展開。

発行日: 2023年12月15日
【続落】
  『はてな』関連は法人向けメディア構築・運用支援の案件増え、個人向けの軟調補う。漫画向け中心に受託サービスの受注拡大。ただ開発人員投資やクラウドサービス利用料の負担増きつく、営業益続落。
【生成ＡＩ】
  『はてなブログ』で機能実装、記事編集画面の入力内容を基

In [5]:
### 業績推移

# KessanFig(
#    code: int, 
#    settlement_type: Literal["通期", "四半期"], 
#    output_target: str = "jupyter",
#    start_settlement_date: date = date(1900, 1, 1),
#    end_settlement_date: date = date(2999, 12, 31)
#)

KFIG = KessanFig(code, "通期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [6]:
# 今季決算予想発表の推移
df = KPL.get_settlement_forcast(code, valuation_date)
df

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64


In [7]:
### 通期業績、業績予想
# def get_df_latest_yearly_performance(code: int, valuation_date: date=date.today()) -> pl.DataFrame:
df = get_df_latest_yearly_performance(code, valuation_date)
df

はてな(3930)の通期決算(評価日：2024年12月13日)


決算期,決算発表日,売上高,営業利益,経常利益,純利益,EPS,1株配当
str,date,i64,i64,i64,i64,f64,f64
"""2024年7月期""",2024-09-13,3309,68,91,62,21.1,0.0


In [8]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()
KPL.get_latest_quater_settlement(code, valuation_date)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
3930,2024-10-31,"""四""",2024-12-13,956,88,92,63,21.4,9.2,1
3930,2024-07-31,"""四""",2024-09-13,931,107,104,71,24.2,11.5,4
3930,2024-04-30,"""四""",2024-05-31,756,-56,-44,-29,-10.9,-7.4,3
3930,2024-01-31,"""四""",2024-03-15,821,19,17,11,3.8,2.3,2
3930,2023-10-31,"""四""",2023-11-30,801,-2,14,9,3.1,-0.2,1
3930,2023-07-31,"""四""",2023-09-13,811,31,42,16,5.5,3.8,4
3930,2023-04-30,"""四""",2023-05-31,771,43,46,29,9.9,5.6,3
3930,2023-01-31,"""四""",2023-02-28,799,52,33,12,4.1,6.5,2


In [9]:
# codeで指定した銘柄のevaluation_dateで指定した時点での最新の年度決算予想に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# from plotly.graph_objects import Figure
# def get_fig_expected_performance_progress_rate_pycharts(code: int, evaluation_date: date=date.today()) -> Figure:

fig1 = get_fig_expected_performance_progress_rate_pycharts(code, valuation_date)
fig1.show()

KeyError: -1

In [ ]:
last_year_valuation_date = date(valuation_date.year-1, valuation_date.month, valuation_date.day)

# codeで指定した銘柄のevaluation_dateで指定した時点での最新の四半期決算発表に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# valuation_dateで指定する日において、当年度の本決算が発表されていない日を指定した場合は前期末第4四半期の進捗率を表示するので、進捗率100%で表示される。
fig2 = get_fig_actual_performance_progress_rate_pycharts(code, last_year_valuation_date, KPL.df, MPL.df)
fig2.show()

In [ ]:
#### valuation_dateに過去日を指定した場合、valuation_dateを含む期の株価推移
#### 答え合わせ

In [ ]:
# 株価推移
# 前期算発表翌営業日～今期決算発表日翌営業日
start_date, end_date = KPL.get_current_settlement_period_by_announcement_date(code, valuation_date, "四")

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>start_date)
start_date = df["date"].to_list()[0]

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>end_date)
if not end_date == date(2999, 12, 31):
    end_date = df["date"].to_list()[0]

PFIG = PricelistFig(code, RevPPL.df, MPL.df, start_date, end_date)
df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")<=valuation_date)
vline_date = df["date"].to_list()[-1]
PFIG.add_vline(vline_date)
PFIG.fig.show()

In [ ]:
# appendix
KPL.df.filter(pl.col("code")==code).filter(pl.col("announcement_date")>=date(2024,1,1))

In [ ]:
PFIG = PricelistFig(code, RevPPL.df, MPL.df, date(2023, 12, 1), date.today())
PFIG.fig.show()

In [ ]:
# 次の決算どうだったか
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date+relativedelta(days=100))
KFIG.add_trace_profits()
KFIG.fig.show()

In [ ]:
from lib_dataprocess import CreditbalancePl

CPL = CreditbalancePl()
CPL.with_columns_margin_ratio()
CPL.df.filter(pl.col("code")==code)